In [6]:
import numpy as np
import cv2
import glob
import math
from tqdm import tqdm
!pip install -q tensorlayer
import tensorlayer as tl
import tensorflow as tf

     |████████████████████████████████| 368kB 9.2MB/s 
     |████████████████████████████████| 3.3MB 13.1MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
base_dir = '3rd College/ProjectX 2020/Temporal SR/eval/'
base_dir = os.path.join('drive', 'My Drive', base_dir)
print(os.listdir(base_dir))

['img_orig', 'img_superslomo', 'videos_half', 'videos_superslomo', 'videos_orig', 'img_bilinear', 'img_half']


In [27]:
'''FETCH LIST OF IMAGES'''
base_dir = '/content/drive/MyDrive/3rd College/ProjectX 2020/Temporal SR/eval/'
gt_dir = ['img_orig/1979/1/*'] #, 'img_orig/1979/2/*', 'img_orig/1979/3/*']
bi_dir = ['img_bilinear/1979/1/*'] #,'img_bilinear/1979/2/*', 'img_bilinear/1979/3/*']
sl_dir = ['img_superslomo/1979/1_2year/*'] #,'img_superslomo/1979/1_3year/*','img_superslomo/1979/1_3year/*']

gt_imgs = []
for dir in gt_dir:
  gt_imgs.extend(glob.glob(base_dir + dir))
gt = []
print('gt images:', len(gt_imgs))
for img in gt_imgs:
  gt.append(cv2.imread(img))

bi_imgs = []
for dir in bi_dir:
  bi_imgs.extend(glob.glob(base_dir + dir))
bi = []
print('bi images:', len(bi_imgs))
for img in bi_imgs:
  bi.append(cv2.imread(img))

sl_imgs = []
for dir in sl_dir:
  sl_imgs.extend(glob.glob(base_dir + dir))
sl = []
print('sl images:', len(sl_imgs))
for img in sl_imgs:
  sl.append(cv2.imread(img))

gt images: 80
bi images: 79
sl images: 78


#### hidden functions

In [ ]:
'''COMPUTE SIM SCORES'''
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

#### Compute

In [28]:
'''Compute scores'''

bi_psnr = []
bi_ssim = []
sl_psnr = []
sl_ssim = []
for i in tqdm(range(len(sl_imgs))):

  original = tf.convert_to_tensor(gt[i], dtype=tf.float32)
  bilinear = tf.convert_to_tensor(bi[i], dtype=tf.float32)
  slomo = tf.convert_to_tensor(sl[i], dtype=tf.float32)

  max = tf.reduce_max(slomo)


  maxes = np.array([255, 255, 255]);
  mins = np.array([0, 0, 0]);
  original  = tf.divide(tf.subtract(original, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));
  bilinear  = tf.divide(tf.subtract(bilinear, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));
  slomo  = tf.divide(tf.subtract(slomo, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));

  bi_psnr.append(tf.image.psnr(original, bilinear, 1.0))
  bi_ssim.append(tf.image.ssim(original, bilinear, 1.0))
  sl_psnr.append(tf.image.psnr(original, slomo, 1.0))
  sl_ssim.append(tf.image.ssim(original, slomo, 1.0))


print(np.mean(np.array(bi_ssim)))
print(np.mean(np.array(sl_ssim)))

100%|██████████| 78/78 [00:02<00:00, 27.85it/s]

0.91805637
0.9511064


In [29]:
print('bilinear ssim', np.mean(np.array(bi_ssim)))
bi_psnr = np.array(bi_psnr)
bi_psnr = np.clip(bi_psnr, 0, 100)
print('bilinear psnr', np.mean(np.array(bi_psnr)))
print('superslomo ssim', np.mean(np.array(sl_ssim)))
print('superslomo psnr', np.mean(np.array(sl_psnr)))

bilinear ssim 0.91805637
bilinear psnr 20.042055
superslomo ssim 0.9511064
superslomo psnr 26.070486


In [9]:
VGG = tl.models.VGG16(pretrained = True, end_with = 'conv2_1')
VGG.train()

[TL] Conv2d conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] LayerList layerlist_1 including layers [conv1_1, conv1_2, pool1, conv2_1]
[TL] Restore pre-trained weights
[TL] Downloading vgg16_weights.npz...



 99% (67513 of 67559) |################# | Elapsed Time: 0:01:22 ETA:   0:00:00

[TL]   Loading weights (3, 3, 3, 64) in conv1_1_W
[TL]   Loading weights (64,) in conv1_1_b
[TL]   Loading weights (3, 3, 64, 64) in conv1_2_W
[TL]   Loading weights (64,) in conv1_2_b
[TL]   Loading weights (3, 3, 64, 128) in conv2_1_W
[TL]   Loading weights (128,) in conv2_1_b


In [30]:
'''Compute scores'''
from sklearn.metrics import mean_squared_error
bi_vgg = []
sl_vgg = []
epoch = 0
for i in tqdm(range(len(sl_imgs))):
  original = tf.convert_to_tensor(gt[i], dtype=tf.float32)
  bilinear = tf.convert_to_tensor(bi[i], dtype=tf.float32)
  slomo = tf.convert_to_tensor(sl[i], dtype=tf.float32)

  maxes = np.array([255, 255, 255]);
  mins = np.array([0, 0, 0]);
  original  = tf.divide(tf.subtract(original, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));
  bilinear  = tf.divide(tf.subtract(bilinear, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));
  slomo  = tf.divide(tf.subtract(slomo, mins), tf.cast(tf.subtract(maxes, mins), tf.float32));


  bi_vgg.append(tl.cost.mean_squared_error(VGG(original), VGG(bilinear), is_mean=True))
  sl_vgg.append(tl.cost.mean_squared_error(VGG(original), VGG(slomo), is_mean=True))
  # print(tl.cost.mean_squared_error(VGG(original), VGG(slomo)))
  
  epoch += 1
  # if epoch > 10:
  #   break


print(np.mean(np.array(bi_vgg)))
print(np.mean(np.array(sl_vgg)))

100%|██████████| 78/78 [00:14<00:00,  5.37it/s]


2375.5513
1258.6643


In [ ]:
print(bi_vgg)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=9.457306>, <tf.Tensor: shape=(), dtype=float32, numpy=18.367828>, <tf.Tensor: shape=(), dtype=float32, numpy=35.069893>, <tf.Tensor: shape=(), dtype=float32, numpy=40.622612>, <tf.Tensor: shape=(), dtype=float32, numpy=55.448387>, <tf.Tensor: shape=(), dtype=float32, numpy=63.800945>, <tf.Tensor: shape=(), dtype=float32, numpy=52.365757>, <tf.Tensor: shape=(), dtype=float32, numpy=63.938023>, <tf.Tensor: shape=(), dtype=float32, numpy=74.23571>, <tf.Tensor: shape=(), dtype=float32, numpy=72.97422>]
